In [18]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import csv
import pandas as pd

import urllib.request,sys,time
from bs4 import BeautifulSoup, UnicodeDammit
import requests
from matplotlib import pyplot as plt
import os

from wordcloud import WordCloud
import pdfplumber
import re

In [30]:
## CNA sample article urls
# url = "https://www.channelnewsasia.com/sustainability/scientists-discover-how-air-pollution-triggers-lung-cancer-2931111"
# url = "https://www.channelnewsasia.com/asia/indonesia-subsidised-fuel-price-increase-micro-businesses-2929006"
url = "https://www.channelnewsasia.com/singapore/nightclub-operator-club-posh-west-palace-entertainment-evade-gst-money-laundering-jailed-penalty-2935436"
# url = "https://cnaluxury.channelnewsasia.com/people/chye-seng-huat-hardware-leon-foo-morning-coffee-machine-206531"

In [53]:
## convert CNA url -> HTML -> CSV
def extract_csv_from_html(url):
    try:
        page=requests.get(url) 
    except Exception as e:    
        error_type, error_obj, error_info = sys.exc_info()      
        print ('ERROR FOR LINK:',url)                     
        print (error_type, 'Line:', error_info.tb_lineno)

    time.sleep(2)
    soup = BeautifulSoup(page.text, "html.parser")


    ## extract article content
    textContent = soup.find_all('div', attrs={'class':'text'})
    paragraphs = []

    for i in textContent:
        para = i.find_all('p')
        for j in para:
            content = j.getText().strip()
            paragraphs.append(content)

    df = pd.DataFrame(paragraphs)
    df.to_csv("backend/data/news.csv", index=False)


message = extract_csv_from_html(url)

In [32]:
## convert cna url -> html -> txt
def extract_txt_from_html(url):
    try:
        page=requests.get(url) 
    except Exception as e:    
        error_type, error_obj, error_info = sys.exc_info()      
        print ('ERROR FOR LINK:',url)                     
        print (error_type, 'Line:', error_info.tb_lineno)

    time.sleep(2)
    soup = BeautifulSoup(page.text, "html.parser")

    ## extract article title
    title = soup.find('h1',attrs={'class':'h1 h1--page-title'}).getText()

    ## extract article content
    textContent = soup.find_all('div', attrs={'class':'text'})
    paragraphs = []

    for i in textContent:
        para = i.find_all('p')
        for j in para:
            content = j.getText().strip()
            paragraphs.append(content)

    single_para = "".join(paragraphs)
    # lines = "\n".join(paragraphs)
    # multi_para = "\n".join(paragraphs)

    f = open("backend/data/article.txt", "w+")
    f.write(single_para)
    f.close()

message = extract_txt_from_html(url)

Text Pre-processing

In [ ]:
# extracts content from csv/txt in /data to an entire paragraph
def extract_text(filename):
    message = ""
    name, extension = os.path.splitext(filename)
    try: 
        if extension == ".csv":   # csv format
            with open("backend/data/{}".format(filename), 'r', encoding="utf8") as f:
                reader = csv.reader(f)
                next(reader, None)  # skip headers
                for row in reader:
                    # print(row)    
                    message += "".join(row)
                # print(sentences)
        elif extension == ".txt":     # txt format
            with open("backend/data/{}".format(filename), 'rb') as f:
                byteString = f.read()
                dammit = UnicodeDammit(byteString, ["utf-8",  "ascii"])
                encoding = dammit.original_encoding
                print("Encoding Type:", encoding)
                message = byteString.decode(encoding, errors='ignore')
                # print(message)
            
        elif extension == '.pdf':
            with pdfplumber.open("backend/data/{}".format(filename)) as pdf:
                for page in pdf.pages:
                    message += page.extract_text() 
    except Exception as err:
            print(err, "occured in"+filename)
    message = message.replace('\n|\\x92"|\\x93|\\x94', ' ') # replace common unicode chars
    return message

# split text > max_length into a list of sentences
def form_text_chunks(document, max_length):
    chunks = []
    sent = ""
    length = 0
    for sentence in document:
        # print(sentence + "\n")
        sentence +=  "."
        length += len(sentence)
        if length < max_length:
            sent += sentence
        else:
            # print(sent + "\n\n")
            chunks.append(sent)
            sent = ""
            length = 0
    if sent:
        chunks.append(sent)
    return chunks

NLP Methods

In [ ]:
# summarize text
def summarize(summarizer, chunks):
    result = ""
    for i in chunks:
        summarized = summarizer(i, max_length=70, min_length=30, do_sample=False)
        # print(summarized[0]["summary_text"])
        result += summarized[0]["summary_text"]
    # print(result)
    return result

def generate_summary(summarizer, filename):
    message = extract_text(filename)
    sentences = message.split('.')
    chunks = form_text_chunks(sentences, 1024)
    # print("chunks:", chunks)
    result = summarize(summarizer, chunks)
    # print(result + "\n")
    while (len(result) > 1200):
        sentences = result.split('.')
        chunks = form_text_chunks(sentences, 1024)
        result = summarize(summarizer, chunks)
    print(result)
    return result
    
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", tokenizer="facebook/bart-large-cnn")
# text = generate_summary(summarizer, "sample_news.csv")
text = generate_summary(summarizer, "article.txt")

In [45]:
def generate_word_list(filename):
    message = extract_text(filename)
    num_words = len(message.split(' '))
    print(num_words)

generate_word_list("article.txt")

Encoding Type: windows-1250
406


In [ ]:
# extract NLP data and export as JSON
def run_chatterbox(text):
    
    # Sentiment

    # Word Cloud

    # Summary

In [46]:
def generate_sentiments(message, model):
    texts = message.split('.')
    # print(texts, "\n\n")
    chunks = form_text_chunks(texts, 512)
    new_df = pd.DataFrame(columns=["Content","Sentiment", "Score"])

    for index in range(len(chunks)):
        preds = model(chunks[index])
        # print(preds)
        pred_sentiment = preds[0]["label"]
        pred_score = preds[0]["score"]

        # write predicted data into df
        new_df.at[index, "Sentiment"] = pred_sentiment
        new_df.at[index, "Score"] = pred_score
        # write text
        new_df.at[index, "Content"] = "".join((chunks[index]))

    # new_df.to_csv("backend/data/results.csv", index=False)
    new_df.to_json("backend/data/results.json")
    results = new_df
    return results

def predict(filename):
    message = extract_text(filename)
    # print(message)
    results = generate_sentiments(message, nlp)
    return results

model = "siebert/sentiment-roberta-large-english"
nlp = pipeline("sentiment-analysis", model=model, tokenizer=model)
results = predict("news.csv")

In [ ]:
def generate_sentiment_graph(results):    
    # visualize the sentiments
    sentiment_counts = results.groupby(['Sentiment']).size()
    print(sentiment_counts)
    fig = plt.figure(figsize=(6,6), dpi=100)
    ax = plt.subplot(111)
    sentiment_counts.plot.pie(ax=ax, autopct='%1.1f%%', startangle=270, fontsize=12, label="")

generate_sentiment_graph(results)

In [ ]:
def generate_word_cloud(results):
    